Used tutorial https://github.com/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb for reference

In [1]:
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from pyspark.sql import types as T

spark = sparknlp.start()

24/11/10 16:42:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
og_data = spark.read.csv("../data_topicmodel.csv", header= True).select(["id", "cleaned_text"])

In [3]:
#Remove sample
og_data.show(5)
sample_data = og_data.sample(0.001)

+-----+--------------------+
|   id|        cleaned_text|
+-----+--------------------+
|hk5r2|i had an appointm...|
|iqimz|i created this si...|
|pfzt5|hello everyone  i...|
|pk714|i grew up with bo...|
|q0q8x|i have to ask whe...|
+-----+--------------------+
only showing top 5 rows



## Preprocessing

In [4]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

In [5]:
tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

In [6]:
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') 

In [7]:
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot']

stopwords = english + time + reddit + topic_specific

from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['normalized']) \
     .setOutputCol('unigrams') \
     .setStopWords(stopwords)

In [8]:
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['normalized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [9]:
pos = PerceptronModel.load("/project/macs40123/spark-jars/pos_anc_en_3.0.0_3.0_1614962126490/")\
      .setInputCols("document", "normalized")\
      .setOutputCol("pos")

In [10]:
from sparknlp.base import Finisher

finisher = Finisher().setInputCols(['unigrams', 'ngrams', 'pos'])

In [11]:
my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          stopwords_cleaner,
          ngrammer,
          pos,
          finisher
      ])

In [12]:
pipelineModel = my_pipeline.fit(sample_data)
processed_data = pipelineModel.transform(sample_data)
processed_data.show(1)

+------+--------------------+--------------------+--------------------+--------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos|
+------+--------------------+--------------------+--------------------+--------------------+
|25fn2l|the biggest obsta...|[biggest, obstacl...|[the, biggest, ob...|[DT, JJS, NN, IN,...|
+------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [13]:
processed_data.columns

['id', 'cleaned_text', 'finished_unigrams', 'finished_ngrams', 'finished_pos']

In [14]:
#data.select('document').show(5, truncate = 100)

In [15]:
#data.select('tokenized').show(1, truncate = 100)

In [16]:
#data.select('normalized').show(1, truncate = 100)

In [17]:
#data.select('lemmatized').show(1, truncate = 1000)

In [18]:
processed_data.select('finished_unigrams').show(10, truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                   finished_unigrams|
+----------------------------------------------------------------------------------------------------+
|[biggest, obstacle, selfimprovement, undeniably, computer, spend, literally, free, using, hobbies...|
|[guys, semester, coming, close, worried, fall, routine, sleeping, working, break, comes, hate, be...|
|[become, akin, mindstate, mindstate, become, unable, even, acknowledge, ability, improve, related...|
|[always, cynical, perspective, humanity, perhaps, feeling, lately, anxiety, convincing, somehow, ...|
|[male, suffered, depression, years, now, saying, never, turned, easy, always, ended, deal, proble...|
|[title, kind, says, slightly, emotional, logical, problems, heart, tends, example, taking, even, ...|
|[shake, angerhurtfrustration, letting, former, friend, years, manipulate

Create pos-tags n-grams that correspond to words n-grams

In [19]:
#Merge POS tags as just one string to be able to take it as a document in the Spark NLP Pipeline
from pyspark.sql import functions as F
pos_as_string = F.udf(lambda x: ' '.join(x), T.StringType())
processed_data = processed_data.withColumn('finished_pos', pos_as_string(F.col('finished_pos')))

In [20]:
processed_data.show(3)

+------+--------------------+--------------------+--------------------+--------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos|
+------+--------------------+--------------------+--------------------+--------------------+
|25fn2l|the biggest obsta...|[biggest, obstacl...|[the, biggest, ob...|DT JJS NN IN PRP$...|
|2m2wpm|hey guys this sem...|[guys, semester, ...|[hey, guys, this,...|UH NNS DT NN VBZ ...|
|32etc8|some become akin ...|[become, akin, mi...|[some, become, ak...|DT VBP NN TO DT N...|
+------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



New pipeline for pos-tags

In [21]:
pos_documentAssembler = DocumentAssembler() \
     .setInputCol('finished_pos') \
     .setOutputCol('pos_document')

pos_tokenizer = Tokenizer() \
     .setInputCols(['pos_document']) \
     .setOutputCol('pos')
     
    
pos_ngrammer = NGramGenerator() \
    .setInputCols(['pos']) \
    .setOutputCol('pos_ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

pos_finisher = Finisher() \
     .setInputCols(['pos', 'pos_ngrams']) \

pos_pipeline = Pipeline() \
     .setStages([pos_documentAssembler,                  
                 pos_tokenizer,
                 pos_ngrammer,  
                 pos_finisher])

processed_data = pos_pipeline.fit(processed_data).transform(processed_data)

In [22]:
processed_data.columns

['id',
 'cleaned_text',
 'finished_unigrams',
 'finished_ngrams',
 'finished_pos',
 'finished_pos_ngrams']

In [23]:
processed_data.show(1)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|25fn2l|the biggest obsta...|[biggest, obstacl...|[the, biggest, ob...|[DT, JJS, NN, IN,...|[DT, JJS, NN, IN,...|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [24]:
def filter_unigrams(finished_unigrams, finished_pos):
    '''Filters individual words based on their POS tag'''
    return [word for word, pos in zip(finished_unigrams, finished_pos)
            if pos in ['JJ', 'NN', 'NNS', 'NNPS']]

udf_filter_unigrams = F.udf(filter_unigrams, T.ArrayType(T.StringType()))

In [25]:
processed_data = processed_data.withColumn('filtered_unigrams_by_pos', udf_filter_unigrams(
                                                   F.col('finished_unigrams'),
                                                   F.col('finished_pos')))

processed_data.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|filtered_unigrams_by_pos|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+
|25fn2l|the biggest obsta...|[biggest, obstacl...|[the, biggest, ob...|[DT, JJS, NN, IN,...|[DT, JJS, NN, IN,...|    [selfimprovement,...|
|2m2wpm|hey guys this sem...|[guys, semester, ...|[hey, guys, this,...|[UH, NNS, DT, NN,...|[UH, NNS, DT, NN,...|    [semester, close,...|
|32etc8|some become akin ...|[become, akin, mi...|[some, become, ak...|[DT, VBP, NN, TO,...|[DT, VBP, NN, TO,...|    [mindstate, unabl...|
|3tcov4|well ive always h...|[always, cynical,...|[well, ive, alway...|[RB, JJ, RB, VBD,...|[RB, JJ, RB, VBD,...|    [cynical, feeling...|
|4rjrkm|im a 20 year old ..

In [26]:
def filter_pos_ngrams(finished_ngrams, finished_pos_tags):
    return [word for word, pos in zip(finished_ngrams, finished_pos_tags) 
            if (len(pos.split('_')) == 2 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS']) \
            or (len(pos.split('_')) == 3 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                  pos.split('_')[2] in ['NN', 'NNS'])]
    
udf_filter_pos_ngrams = F.udf(filter_pos_ngrams, T.ArrayType(T.StringType()))

In [27]:
processed_data = processed_data.withColumn('filtered_ngrams_by_pos',
                       udf_filter_pos_ngrams(F.col('finished_ngrams'),
                                             F.col('finished_pos_ngrams')))

In [28]:
processed_data.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+----------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|filtered_unigrams_by_pos|filtered_ngrams_by_pos|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+----------------------+
|25fn2l|the biggest obsta...|[biggest, obstacl...|[the, biggest, ob...|[DT, JJS, NN, IN,...|[DT, JJS, NN, IN,...|    [selfimprovement,...|  [free_time, tv_se...|
|2m2wpm|hey guys this sem...|[guys, semester, ...|[hey, guys, this,...|[UH, NNS, DT, NN,...|[UH, NNS, DT, NN,...|    [semester, close,...|  [ill_fall, become...|
|32etc8|some become akin ...|[become, akin, mi...|[some, become, ak...|[DT, VBP, NN, TO,...|[DT, VBP, NN, TO,...|    [mindstate, unabl...|  [become_akin, bec...|
|3tcov4|well ive always h...

In [29]:
processed_data.select('id','filtered_ngrams_by_pos').show(10, truncate=100)

+------+----------------------------------------------------------------------------------------------------+
|    id|                                                                              filtered_ngrams_by_pos|
+------+----------------------------------------------------------------------------------------------------+
|25fn2l|[free_time, tv_series, complete_change, entire_lifestyle, lifestyle_ive, internet_hobby, hobby_th...|
|2m2wpm|             [ill_fall, become_stagnant, stay_productive, productive_thanks, stay_productive_thanks]|
|32etc8|[become_akin, become_unable, own_self, self_ability, chakra_zen, zen_state, be_wrong, please_dire...|
|3tcov4|[cynical_perspective, theres_nothing, nothing_new, last_few, few_months, general_thats, have_frie...|
|4rjrkm|[year_old, old_male, male_ive, past_years, happy_kid, kid_yrs, yrs_old, naive_soul, high_school, ...|
|8it6z4|                                             [title_kind, im_someone, someone_whos, im_someone_whos]|
|8sbqyi|[c

In [30]:
processed_data.select('id','filtered_unigrams_by_pos').show(10, truncate=100)

+------+----------------------------------------------------------------------------------------------------+
|    id|                                                                            filtered_unigrams_by_pos|
+------+----------------------------------------------------------------------------------------------------+
|25fn2l|[selfimprovement, spend, free, soccer, tv, series, literally, giving, tried, hobbies, replace, in...|
|2m2wpm|                                            [semester, close, working, comes, stagnant, break, stay]|
|32etc8|                                            [mindstate, unable, related, akin, state, wrong, please]|
|3tcov4|[cynical, feeling, convincing, somehow, lately, slowly, developed, mild, say, friends, random, de...|
|4rjrkm|[male, depression, years, now, saying, easy, deal, problems, friends, started, never, friendsi, m...|
|8it6z4|              [kind, says, heart, tends, example, anymore, memories, telling, slowly, slowly, heart]|
|8sbqyi|[a

In [31]:
processed_data.columns

['id',
 'cleaned_text',
 'finished_unigrams',
 'finished_ngrams',
 'finished_pos',
 'finished_pos_ngrams',
 'filtered_unigrams_by_pos',
 'filtered_ngrams_by_pos']

In [32]:
#Now that POS was done, lemmatization makes more sense at this point

#Merge tokens as just one string to be able to take it as a document in the new Pipeline
from pyspark.sql import functions as F
tokens_as_string = F.udf(lambda x: ' '.join(x), T.StringType())
processed_data = processed_data.withColumn('joined_tokens', tokens_as_string(F.col('filtered_unigrams_by_pos')))

last_documentAssembler = DocumentAssembler() \
     .setInputCol('joined_tokens') \
     .setOutputCol('joined_document')

last_tokenizer = Tokenizer() \
     .setInputCols(['joined_document']) \
     .setOutputCol('tokenized')
     
lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("tokenized")\
      .setOutputCol("lemmatized")

#Delete these tokens that remained from the lemmatizer model and topic's n grams
last_stopwords = ["_", "self_improvement"]

last_stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('cleaned_unigrams') \
     .setStopWords(last_stopwords)

last_finisher = Finisher() \
     .setInputCols(['cleaned_unigrams']) \

last_pipeline = Pipeline() \
     .setStages([last_documentAssembler,                  
                 last_tokenizer,
                 lemmatizer,
                 last_stopwords_cleaner,
                 last_finisher])

final_data = last_pipeline.fit(processed_data).transform(processed_data)


Create one column merging unigrams and ngrams

In [33]:
from pyspark.sql.functions import concat
final_data = final_data.withColumn('final', concat(F.col('finished_cleaned_unigrams'), \
                                                   F.col('filtered_ngrams_by_pos')))\
                                                   .select('id','cleaned_text','final')
                                                                                                                          

In [34]:
final_data.select('final').show(50, truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                               final|
+----------------------------------------------------------------------------------------------------+
|[selfimprovement, spend, free, soccer, tv, series, literally, give, try, hobbies, replace, intern...|
|[semester, close, working, come, stagnant, break, stay, ill_fall, become_stagnant, stay_productiv...|
|[mindstate, unable, related, akin, state, wrong, please, become_akin, become_unable, own_self, se...|
|[cynical, feeling, convince, somehow, lately, slowly, develop, mild, say, random, develop, experi...|
|[male, depression, year, now, saying, easy, deal, problem, start, never, friendsi, mind, loud, en...|
|[kind, say, heart, tend, example, anymore, memory, tell, slowly, slowly, heart, title_kind, im_so...|
|[angerhurtfrustration, let, year, crazy, illogical, memory, whirring, se

In [35]:
final_data.columns

['id', 'cleaned_text', 'final']

## Vectorization

In [36]:
#Apply TF-IDF filtering
tfizer = CountVectorizer(inputCol='final', outputCol='tf_features', minDF=0.01, maxDF=0.70)
tf_model = tfizer.fit(final_data)
tf_result = tf_model.transform(final_data)

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

## LDA

In [37]:
num_topics = 10
max_iter = 100

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features', seed=2503)
lda_model = lda.fit(tfidf_result)
lda_model.setSeed(2503)

24/11/10 16:43:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/11/10 16:43:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


LocalLDAModel: uid=LDA_a87aaf592677, k=10, numFeatures=351

In [38]:
vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [39]:
num_top_words = 15

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)

+-----+------------------------------------------------------------------------------------------+
|topic|                                                                                topicWords|
+-----+------------------------------------------------------------------------------------------+
|    0|[other_people, tell, exercise, skill, now, few_months, seem, little, deep, see, always,...|
|    1|[social_media, sleep, show, college, enjoy, video, social, sure, check, reason, cause, ...|
|    2|[long, great, book, day, need, read, create, try, whatever, honest, hope, truth, sorry,...|
|    3|[sometimes, come, making, alone, though, instead, struggle, talk, be_honest, give, scho...|
|    4|[live, part, other, habit, choice, healthy, never, good, find, opinion, feeling, spend,...|
|    5|[job, one, mind, eat, ever, definitely, avoid, issue, easy, take, mental_health, anymor...|
|    6|[video_games, self_improvement, happen, back, turn, okay, plan, eventually, hour, game,...|
|    7|[le

# REMEMBER TO CHANGE DATASET TO COMPLETE ONE